In [71]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

In [72]:
with open('train_low_None_.pickle', 'rb') as f:
    Data = pickle.load(f)
Data.tail()

,grid_0.0,grid_0.001,grid_0.002,grid_0.003,grid_0.004,grid_0.005,grid_0.006,grid_0.007,grid_0.008,grid_0.009000000000000001,...,grid_0.992,grid_0.993,grid_0.994,grid_0.995,grid_0.996,grid_0.997,grid_0.998,grid_0.999,grid_1.0,label
795,0.105068,0.060416,0.049944,0.099071,0.041751,0.120240,0.105756,0.117128,0.070197,0.086788,...,0.100335,0.073304,0.085145,0.034842,0.100622,0.061166,0.076784,0.071702,0.071898,TILVUF
796,0.053610,0.035839,0.048673,0.056148,0.083227,0.043436,0.007687,0.028381,0.075905,0.063874,...,0.026327,0.024053,0.060030,0.020799,0.051219,0.040494,0.040977,0.028633,0.045080,TILVUF
797,0.008310,0.010341,0.005085,0.012205,0.005969,0.012394,0.017779,0.013828,0.020269,0.014279,...,0.012832,0.011732,0.015073,0.017813,0.018217,0.046443,0.022640,0.008357,0.008169,TILVUF
798,0.048485,0.066823,0.081387,0.088221,0.001269,0.035857,0.041821,0.092311,0.030829,0.056996,...,0.074983,0.067087,0.085165,0.061939,0.073722,0.096549,0.098253,0.033827,0.050147,TILVUF
799,0.069574,0.084375,0.096658,0.065596,0.051090,0.092272,0.076667,0.050205,0.068758,0.045231,...,0.071965,0.080382,0.091502,0.066572,0.072682,0.082091,0.106418,0.054214,0.063668,TILVUF


In [73]:
Data1 = Data.loc[Data['label'] == 'ZIF-8']
Data2 = Data.loc[Data['label'] == 'ZIF-67']
Data3 = Data.loc[Data['label'] == 'ZIF-90']
Data4 = Data.loc[Data['label'] == 'ZIF-71']
Data5 = Data.loc[Data['label'] == 'ZIF-65']
Data6 = Data.loc[Data['label'] == 'ZIF-11']
Data7 = Data.loc[Data['label'] == 'DUT-23']
Data8 = Data.loc[Data['label'] == 'AFIXES']
Data9 = Data.loc[Data['label'] == 'DUT-10']
Data10 = Data.loc[Data['label'] == 'HUYKUH']
Data11 = Data.loc[Data['label'] == 'LUYNAU']
Data12 = Data.loc[Data['label'] == 'JOSNAG01']
Data13 = Data.loc[Data['label'] == 'F-MOF-11']
Data14 = Data.loc[Data['label'] == 'AFIXES']
Data15 = Data.loc[Data['label'] == 'JOKYOX10']
Data16 = Data.loc[Data['label'] == 'ZIF-12']
Data17 = Data.loc[Data['label'] == 'JOKYOX10']
Data18 = Data.loc[Data['label'] == 'YUTGEZ']
Data19 = Data.loc[Data['label'] == 'PIZJEN']
Data20 = Data.loc[Data['label'] == 'ZIF-74']

Data_20 = pd.concat([Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,Data9,Data10,Data11,Data12,Data13,Data14,Data15,Data16,Data17,Data18,Data19,Data20,])

In [74]:
Data_20.tail()
Data_20.shape

(16000, 1002)

In [75]:
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, Flatten
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.layers import Input, Add, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D
from keras.initializers import glorot_uniform
from keras.models import Model, load_model


In [76]:
import os
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [77]:
idx = random.sample(range(16000), 3200) # 80% for training随机抽取范围内20%的数字
all_index = list(np.setdiff1d(range(16000), idx))#取抽取出来和原数字不同的数，就是80% train

# Getting X_train, X_val, y_train and scale them
X_train = Data.iloc[all_index,:].drop('label', axis = 1)
X_val = Data.iloc[idx,:].drop('label', axis = 1)
y_train = pd.get_dummies(Data.iloc[all_index,:]['label'])
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)

(12800, 1001)
(3200, 1001)
(12800, 20)


In [78]:
names = y_train.columns
mapping = {}
i = 0
for n in names:
    mapping[i] = n
    i+=1

# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [79]:
# Construct CNN layers
# 浅一些的vgg 16
cnn_model = Sequential()
cnn_model.add(Conv1D(64, 3, strides = 1, activation = 'relu', input_shape = (1001, 1)))
cnn_model.add(Conv1D(64, 3, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(128, 3, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(128, 3, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(256, 3, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(256, 3, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(512, 3, strides = 1, activation = 'relu'))
#cnn_model.add(Conv1D(512, 3, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))

#cnn_model.add(Dense(4096, activation = 'relu'))
#cnn_model.add(Dense(4096, activation = 'relu'))

cnn_model.add(Dense(20, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [54]:
# Construct CNN layers
# LeNet-5
cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (1001, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(20, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [80]:
# Fitting CNN model
cnn_model.fit(X_train.reshape(12800, 1001, 1), y_train.values, batch_size = 64, epochs = 100, verbose = 1)

# Validating the CNN modela
pred = cnn_model.predict(X_val.reshape(3200, 1001, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)

# Create category mapping dictionary
y_val = Data.iloc[idx,:]['label']
    
# Calculating accuracy
predicted = np.vectorize(mapping.get)(out)
print('Accuracy: '+ str(round(np.sum(predicted == y_val)/2000,2) * 100) + '%')

Epoch 1/100
12800/12800 [==============================] - 7s 515us/step - loss: 2.1406 - acc: 0.9052
Epoch 2/100
12800/12800 [==============================] - 6s 430us/step - loss: 1.9566 - acc: 1.0000
Epoch 3/100
12800/12800 [==============================] - 6s 430us/step - loss: 1.9502 - acc: 1.0000
Epoch 4/100
12800/12800 [==============================] - 5s 428us/step - loss: 1.9488 - acc: 1.0000
Epoch 5/100
12800/12800 [==============================] - 6s 430us/step - loss: 1.9478 - acc: 1.0000
Epoch 6/100
12800/12800 [==============================] - 6s 430us/step - loss: 1.9473 - acc: 1.0000
Epoch 7/100
12800/12800 [==============================] - 5s 429us/step - loss: 1.9474 - acc: 1.0000
Epoch 8/100
12800/12800 [==============================] - 5s 429us/step - loss: 1.9468 - acc: 1.0000
Epoch 9/100
12800/12800 [==============================] - 6s 430us/step - loss: 1.9468 - acc: 1.0000
Epoch 10/100
12800/12800 [==============================] - 6s 430us/step - loss: 

KeyboardInterrupt: 

In [81]:
with open('test__None_.pickle', 'rb') as f:
    Test = pickle.load(f)

Test.head()

,grid_0.0,grid_0.001,grid_0.002,grid_0.003,grid_0.004,grid_0.005,grid_0.006,grid_0.007,grid_0.008,grid_0.009000000000000001,...,grid_0.992,grid_0.993,grid_0.994,grid_0.995,grid_0.996,grid_0.997,grid_0.998,grid_0.999,grid_1.0,label
0,0.033514,0.020624,0.027069,0.023330,0.000000,0.046661,0.039185,0.053880,0.034029,0.057747,...,0.034544,0.065352,0.048982,0.055169,0.043825,0.048208,0.050271,0.037638,0.054138,ZIF-67
1,0.034760,0.042744,0.025364,0.007046,0.096761,0.002820,0.030059,0.021604,0.025364,0.033349,...,0.568339,0.564113,0.563644,0.537812,0.566462,0.549084,0.538280,0.569748,0.620950,ZIF-67
2,0.032539,0.046020,0.042301,0.044392,0.039976,0.062521,0.064148,0.049273,0.067402,0.057640,...,0.038581,0.042765,0.022313,0.022429,0.041836,0.031144,0.028820,0.036722,0.043695,ZIF-67
3,0.029431,0.080294,0.040947,0.061420,0.055022,0.084453,0.068458,0.102047,0.063980,0.064299,...,0.066539,0.047345,0.050544,0.031030,0.039667,0.059501,0.039667,0.045106,0.047345,ZIF-67
4,0.038178,0.038881,0.037756,0.039795,0.033115,0.037334,0.038178,0.034873,0.033397,0.039303,...,0.013148,0.009843,0.012304,0.013007,0.015116,0.010617,0.010898,0.008156,0.011742,ZIF-8


In [82]:
Test.shape

(14, 1002)

In [83]:
X_test = scaler.transform(Test.iloc[:,:-1]) #transform数据处理成正态分布,shape(14, 1001)
print(X_test.shape)
pred = cnn_model.predict(X_test.reshape(14, 1001, 1)) #pred.shape(14, 20)
print(pred.shape)
out = [np.argmax(p) for p in pred]
print(out)
out = np.array(out)
print(out)
predicted = np.vectorize(mapping.get)(out)
print(predicted)

(14, 1001)
(14, 20)
[4, 2, 4, 4, 2, 4, 4, 4, 18, 4, 4, 4, 18, 18]
[ 4  2  4  4  2  4  4  4 18  4  4  4 18 18]
['JARMEU' 'CIDNIM' 'JARMEU' 'JARMEU' 'CIDNIM' 'JARMEU' 'JARMEU' 'JARMEU'
 'ZIF-8' 'JARMEU' 'JARMEU' 'JARMEU' 'ZIF-8' 'ZIF-8']


In [84]:
preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:5]:
    P.append(mapping[i])
  preds.append(P)

In [85]:
for p in preds:
  print(p)

['JARMEU', 'CIDNIM', 'PIZJEN', 'ZIF-75', 'AENCESE10']
['CIDNIM', 'ZIF-64', 'JARMEU', 'PIZJEN', 'ZIF-2']
['JARMEU', 'CIDNIM', 'PIZJEN', 'ZIF-75', 'WIWQUO']
['JARMEU', 'CIDNIM', 'PIZJEN', 'ZIF-75', 'ZIF-2']
['CIDNIM', 'JARMEU', 'PIZJEN', 'ZIF-8', 'WIWQUO']
['JARMEU', 'CIDNIM', 'PIZJEN', 'ZIF-75', 'AENCESE10']
['JARMEU', 'CIDNIM', 'PIZJEN', 'ZIF-75', 'AENCESE10']
['JARMEU', 'CIDNIM', 'PIZJEN', 'AENCESE10', 'ZIF-64']
['ZIF-8', 'PIZJEN', 'JARMEU', 'ZIF-64', 'AENCESE10']
['JARMEU', 'CIDNIM', 'PIZJEN', 'AENCESE10', 'ZIF-75']
['JARMEU', 'CIDNIM', 'PIZJEN', 'AENCESE10', 'ZIF-2']
['JARMEU', 'CIDNIM', 'PIZJEN', 'AENCESE10', 'ZIF-2']
['ZIF-8', 'PIZJEN', 'ZIF-64', 'RASGAT', 'JOKYOX']
['ZIF-8', 'PIZJEN', 'JARMEU', 'ZIF-64', 'AENCESE10']


In [43]:
Test.label

0     ZIF-67
1     ZIF-67
2     ZIF-67
3     ZIF-67
4      ZIF-8
5      ZIF-8
6      ZIF-8
7     ZIF-90
8     ZIF-71
9      ZIF-8
10    ZIF-90
11    ZIF-90
12    ZIF-71
13    ZIF-71
Name: label, dtype: object